In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
from crewai import Agent, Task, Crew


In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)


In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)


In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool


In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()


In [8]:
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"


In [9]:
sentiment_analysis_tool = SentimentAnalysisTool()


In [10]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)


In [11]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)


In [12]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True,
)


In [13]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To begin the task, I need to gather initial information about DeepLearningAI, including key decision-makers, recent business developments, and potential needs. This will be crucial for tailoring our engagement strategy effectively.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & 

In [14]:
from IPython.display import Markdown
Markdown(result)


---
**Subject: Empowering DeepLearningAI's Future with Advanced AI Solutions**

**Email 2: Subject: Unlock New Possibilities in AI Learning**
Dear Andrew,

I hope this message finds you well. I’m reaching out to congratulate you on DeepLearningAI’s recent strides in generative AI applications. It’s inspiring to see your commitment to pushing the boundaries of AI education and making these advanced tools accessible and practical.

At CrewAI, we recognize the challenges and opportunities that come with such innovative endeavors. Our AI Integration Solutions are designed to seamlessly blend with educational platforms like yours, enhancing content delivery and enriching the learning experience through personalized educational journeys.

Given your recent product launches, I believe there's a tremendous opportunity for us to collaborate. Our expertise in upgrading legacy systems and integrating cutting-edge AI models could significantly elevate DeepLearningAI’s offerings. We can provide the technological backbone that allows you to scale these solutions efficiently and effectively.

Furthermore, our specialized Talent Acquisition and Training Programs could be the key to bridging the talent gap in AI and machine learning, ensuring that your team remains at the forefront of AI application.

I would love to discuss how CrewAI can partner with DeepLearningAI to turn these possibilities into realities. Are you available for a brief call next week to explore this further?

Looking forward to the opportunity to collaborate.

Warm regards,

[Your Name]
Lead Sales Representative, CrewAI
[Your Contact Information]
---

**Subject: Strengthening DeepLearningAI's Legacy with Next-Gen AI Solutions**

Hello Andrew,

It’s exciting to see DeepLearningAI’s recent achievements in generative AI technologies. Your vision for a more AI-fluent world resonates deeply with us at CrewAI, where we are dedicated to enhancing AI integration across various sectors.

I understand that integrating such advanced technologies into existing platforms presents unique challenges, from system compatibility to user adaptability. Our Legacy System Modernization service is tailored to help educational leaders like DeepLearningAI overcome these hurdles, ensuring that your infrastructure is not just updated but optimized for future advancements.

Could we set up a time to discuss how CrewAI’s solutions can align with your strategic goals and help overcome any technical challenges you might be facing?

Best regards,
[Your Name]
Lead Sales Representative, CrewAI

**Email 3: Subject: A Strategic Partnership in AI Education**
Dear Andrew,
It's clear that both DeepLearningAI and CrewAI share a common goal: to transform the landscape of AI education. Our analytics tools can provide invaluable insights into course effectiveness and learner preferences, enabling you to tailor educational content more precisely. Imagine the impact we could have through a strategic partnership that leverages the strengths of both our platforms.
I am looking forward to exploring this potential with you.
Sincerely,
[Your Name]
Lead Sales Representative, CrewAI
[Your Contact Information]
---

**Subject: Partnering for Innovation: Tailored AI Training Programs for DeepLearningAI**

Dear Andrew,

Congratulations on your recent launch of innovative generative AI applications at DeepLearningAI. As AI continues to evolve, the need for specialized skills to manage and leverage these technologies becomes increasingly critical.

At CrewAI, we specialize in developing customized training modules that can accelerate skill acquisition in AI and machine learning, tailored specifically to the needs of your team. Our collaborative approach ensures that these programs not only address skill gaps but also empower your team to manage and innovate with the new technologies you are developing.

I’m keen to explore how we can assist DeepLearningAI in enhancing its educational impact through targeted training and development. Would you be available for a discussion on how we can support your goals in this area?

Thank you for considering this opportunity for synergy. I look forward to your thoughts.

Sincerely,

[Your Name]
Lead Sales Representative, CrewAI
[Your Contact Information]
---

These emails are crafted to address the specific needs and developments at DeepLearningAI, proposing clear solutions and inviting further discussion to foster a collaborative relationship. Each email is tailored to resonate with DeepLearningAI’s mission and recent activities, emphasizing mutual growth and shared values in advancing AI education.